In [3]:
# Activate packages in the manifest of this directory
using Pkg; Pkg.activate(@__DIR__)

  Activating project at `c:\Users\rcrocker\Documents\Github\ReefBiodiversityAccountingModellingExamples\ADRIA.jl runs`


# Running ADRIA for biodiversity accounting modelling
This script demonstrates running intervention scenarios in ADRIA.jl to create a biodiversity accounting table of a series of
intervention scenarios.

## Load packages
First load the required packages into the Julia environment, including ADRIA.

In [4]:
using ADRIA
using DataFrames, YAXArrays # For manipulating tables and data cubes

## Load ADRIA Domain 
First, load the ADRIA domain, which contains all necessary spatial, environmental and ecological data for the selection of reefs being modelled.

In [5]:
path_to_domain = "C:\\Users\\rcrocker\\Documents\\Datapackages\\Moore_2025-03-18_v070_rc1" # Insert filepath to ADRIA datapackage here

# Load ADRIA domain (e.g. Moore)
dom = ADRIA.load_domain(
    path_to_domain,
    "45"
)

┌ Warning: Cyclone mortality uses 6 functional groups. ADRIA uses 5.
│ Skipping first functional group.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\ExtInterface\ADRIA\Domain.jl:203


Domain: Moore_2025-03-18_v070_rc1

Number of locations: 334
Location data file: C:\Users\rcrocker\Documents\Datapackages\Moore_2025-03-18_v070_rc1\spatial\Moore_2025-03-18_v070_rc1.gpkg
Connectivity file: C:\Users\rcrocker\Documents\Datapackages\Moore_2025-03-18_v070_rc1\connectivity/
DHW file: C:\Users\rcrocker\Documents\Datapackages\Moore_2025-03-18_v070_rc1\DHWs\dhwRCP45.nc
Wave file: C:\Users\rcrocker\Documents\Datapackages\Moore_2025-03-18_v070_rc1\waves\wave_RCP45.nc
Cyclone mortality file:
Timeframe: 2025 - 2099

Model Specification:
- Parameters: 320
- Number of constants: 24



## Define scenario parameters

Here we define key parameters for the scenarios we want to run. These include DHW enhancement levels and coral deployment volumes, as well as the start year and number of years to seed and the criteria to use when selecting seeding sites.

Parameters are set constant using `fix_factor` so that the main source of uncertainty is due to sampling DHW/climate model scenarios. Below describes some key ADRIA parameters which can be set:

- `a_adapt` : DHW adaptation level.
- `N_seed_TA`, `N_seed_CA`, `N_seed_SM` : number of Tabular Acropora, Corymbose Acropora and Small Massives to outplant each year.
- `seed_year_start` : When to start seeding.
- `seed_deployment_freq` : How frequently to reassess which sites to seed at. If =1, this is decided in the first year and not changed.
- `plan_horizon` : Number of years to look into the future for heat stress criteria when selecting sites.
- `seed_years` : Number of years to seed over.
- `min_iv_locations` : Number of locations to seed at.
- `seed_heat_stress` : Weighting for the heat stress criteria when choosing sites for seeding.
- `seed_depth` : Weighting for the site depth criteria when choosing sites for seeding.
- `seed_coral_cover` : Weighting for the coral available space criteria when choosing sites for seeding.

In [ ]:
# Set adaptation levels to run
aadpt_dhw_range = [5, 10, 15]
# Set coral deployment levels to run
n_corals_range = [200000, 500000, 1000000]
# Year index to start seeding
seed_start = 1
# Frequency with which to re-evaluate seeding locations (1 implies evaluate at the start and don't change)
seed_dep_freq = 1
# Number of years ahead to evaluate in terms of heat stress when selecting seeding sites
plan_window = 10
# Number of years to seed from first seeding year
n_seed_years = 10
# Minimum number of locations to choose to seed at
min_locations = 15.0
# Weighting to giving to heat stress criteria when selecting locations
heat_stress_weight = 1.0
# Weightings to give to give to incoming and outgoing connectivity criteria when selecting locations
in_conn_weight = 0.0
out_conn_weight = 0.0
# Weighting to give to give to depth criteria when selecting locations
depth_weight = 1.0
# Weighting to give to give to available space for coral cover criteria when selecting locations
space_weight = 1.0

## Construct the scenario dataframe

Create a basic scenario dataframe. This is a table of sampled parameters for every parameter defined in ADRIA. We then loop through each of the intervention parameters we want to include in our runs, set these parameters contstant, and resample. The resulting dataframes are appended to this initial sampled dataframe to create a dataframe of all the parameter samples we want to run in ADRIA. 

In [ ]:
# Extract parameters related to the coral model to hold these constant
coral_model_factors = ADRIA.model_spec(dom).fieldname[ADRIA.model_spec(dom).component.=="Coral"]

# Sample 2 scenarios only to get scenario datframe structure
scens = ADRIA.sample(dom, 2^1)

# Loop through intervention scenario settings and sample
for aapt in aadpt_dhw_range
    for dep_eff in n_corals_range
        ADRIA.set_factor_bounds!(dom, :guided, ("counterfactual", "COCOSO",)) # Only run counterfactuals, unguided and guided with 1st algorithm
        ADRIA.fix_factor!(dom, coral_model_factors) # Fix coral model factors
        ADRIA.fix_factor!(dom;
            a_adapt = aapt, # Set adaptation DHW level
            N_seed_TA = dep_eff/3, # Set coral deployment level (split evenly across species)
            N_seed_CA = dep_eff/3,
            N_seed_SM = dep_eff/3,
            seed_year_start = seed_start, # Start seeding in 2025
            shade_year_start = 0,
            fog_year_start = 0,
            seed_deployment_freq = seed_dep_freq, # Choose sites to deploy at in year one and deploy at the same sites
            fog_deployment_freq = 0,
            shade_deployment_freq = 0,
            cyclone_mortality_scenario = 0,
            wave_scenario = 0,
            plan_horizon = plan_window, # Look 10 years into the future (for DHWs) to select sites
            seed_years = n_seed_years, # Seed for a total of 10 years
            fogging = 0.0, # No fogging or SRM
            SRM = 0.0,
            fog_years = 0.0,
            shade_years = 0.0,
            min_iv_locations = min_locations, # Choose a maximum of 15 locations to seed at
            seed_heat_stress = heat_stress_weight, # Use heat stress to choose sites
            seed_wave_stress = 0.0,
            seed_in_connectivity = in_conn_weight,
            seed_out_connectivity = out_conn_weight,
            seed_depth = depth_weight, # Use depth to choose sites
            seed_coral_cover = space_weight) # Use available space to choose sites

        # Sample to get scenario dataframe
        scens_temp = ADRIA.sample(dom, 2^8)

        # Append scenario dataframe to others so all scenarios can be run at once
        global scens = vcat(scens, scens_temp)

    end
end
# Remove initial 2 scenarios which were sampled without settings
scens = scens[3:end,:]


┌ Warning: Non-unique samples created: 0.391% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 0.391% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 0.781% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 0.391% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 0.391% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 1.172% of the samples are duplicates.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\io\sampling.jl:72
┌ Warning: Non-unique samples created: 0.781% of the samples are duplicates.
└ @ A

Row,dhw_scenario,wave_scenario,cyclone_mortality_scenario,guided,N_seed_TA,N_seed_CA,N_seed_SM,min_iv_locations,fogging,SRM,a_adapt,seed_years,shade_years,fog_years,plan_horizon,seed_deployment_freq,fog_deployment_freq,shade_deployment_freq,seed_year_start,shade_year_start,fog_year_start,seed_heat_stress,seed_wave_stress,seed_in_connectivity,seed_out_connectivity,seed_depth,seed_coral_cover,seed_cluster_diversity,seed_geographic_separation,fog_heat_stress,fog_wave_stress,fog_in_connectivity,fog_out_connectivity,fog_depth,fog_coral_cover,fog_cluster_diversity,fog_geographic_separation,depth_min,depth_offset,heritability,tabular_Acropora_1_1_mean_colony_diameter_m,tabular_Acropora_1_1_linear_extension,tabular_Acropora_1_1_growth_rate,tabular_Acropora_1_1_fecundity,tabular_Acropora_1_1_mb_rate,tabular_Acropora_1_1_bin_ub,tabular_Acropora_1_1_dist_mean,tabular_Acropora_1_1_dist_std,tabular_Acropora_1_2_mean_colony_diameter_m,tabular_Acropora_1_2_linear_extension,tabular_Acropora_1_2_growth_rate,tabular_Acropora_1_2_fecundity,tabular_Acropora_1_2_mb_rate,tabular_Acropora_1_2_bin_ub,tabular_Acropora_1_2_dist_mean,tabular_Acropora_1_2_dist_std,tabular_Acropora_1_3_mean_colony_diameter_m,tabular_Acropora_1_3_linear_extension,tabular_Acropora_1_3_growth_rate,tabular_Acropora_1_3_fecundity,tabular_Acropora_1_3_mb_rate,tabular_Acropora_1_3_bin_ub,tabular_Acropora_1_3_dist_mean,tabular_Acropora_1_3_dist_std,tabular_Acropora_1_4_mean_colony_diameter_m,tabular_Acropora_1_4_linear_extension,tabular_Acropora_1_4_growth_rate,tabular_Acropora_1_4_fecundity,tabular_Acropora_1_4_mb_rate,tabular_Acropora_1_4_bin_ub,tabular_Acropora_1_4_dist_mean,tabular_Acropora_1_4_dist_std,tabular_Acropora_1_5_mean_colony_diameter_m,tabular_Acropora_1_5_linear_extension,tabular_Acropora_1_5_growth_rate,tabular_Acropora_1_5_fecundity,tabular_Acropora_1_5_mb_rate,tabular_Acropora_1_5_bin_ub,tabular_Acropora_1_5_dist_mean,tabular_Acropora_1_5_dist_std,tabular_Acropora_1_6_mean_colony_diameter_m,tabular_Acropora_1_6_linear_extension,tabular_Acropora_1_6_growth_rate,tabular_Acropora_1_6_fecundity,tabular_Acropora_1_6_mb_rate,tabular_Acropora_1_6_bin_ub,tabular_Acropora_1_6_dist_mean,tabular_Acropora_1_6_dist_std,tabular_Acropora_1_7_mean_colony_diameter_m,tabular_Acropora_1_7_linear_extension,tabular_Acropora_1_7_growth_rate,tabular_Acropora_1_7_fecundity,tabular_Acropora_1_7_mb_rate,tabular_Acropora_1_7_bin_ub,tabular_Acropora_1_7_dist_mean,tabular_Acropora_1_7_dist_std,corymbose_Acropora_2_1_mean_colony_diameter_m,corymbose_Acropora_2_1_linear_extension,corymbose_Acropora_2_1_growth_rate,corymbose_Acropora_2_1_fecundity,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,146.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0375,0.00609456,0.487565,0.0,0.4,0.05,3.75161,2.90443,0.0625,0.0107184,0.857472,0.0,0.24,0.075,3.75161,2.90443,0.0875,0.0255149,2.04119,0.0,0.195,0.1,3.75161,2.90443,0.15,0.0507988,1.01598,4.38605e5,0.24,0.2,3.75161,2.90443,0.3,0.0945091,0.945091,6.46622e5,0.15,0.4,3.75161,2.90443,0.7,0.168505,0.561683,1.03924e6,0.14,1.0,3.75161,2.90443,1.25,0.0,0.0,1.43791e6,0.14,1.5,3.75161,2.90443,0.0375,0.007


## Run scenarios

Run the scenarios in the dataframe created and find the filename where the results will be saved.


In [8]:
rs = ADRIA.run_scenarios(dom, scens, ["45"]) # Run scenarios
results_location = ADRIA.result_location(rs) # Get filepath to resultset

┌ Warning: Could not find config.toml file.
│ Applying default configuration and saving results to 'Outputs' in current directory.
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\utils\setup.jl:21
┌ Info: Running 2304 scenarios over 1 RCPs: ["45"]
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\scenario.jl:131
┌ Info: Setting up Result Set
└ @ ADRIA C:\Users\rcrocker\Documents\Github\ADRIA.jl\src\scenario.jl:138
Running 2304 scenarios for RCP 45 100%|██████████████████| Time: 0:39:54K


"./Outputs/Moore_2025-03-18_v070_rc1__RCPs_45__2025-07-08_14_21_59_463"